## 1. 资产收益预估统计模型的总体构造




使用机器学习模型进行资产回报的预估，建立如下的加法模型
$$
r_{i, t+1} = E_t(r_{i, t+1}) + \epsilon_{i, t+1} \tag{1}
$$
其中$$
E_t(r_{i, t+1}) = g^*(z_{i, t+1}) \tag{2}
$$

$i = 1, 2, ... N_t$为股票id， $t = 1, 2,...,T$为月份序号。
即第i个股票第t+1期的收益为期望收益 + 误差项。

模型中的期望函数g是因子$z_{i, t+1}$的函数， 模型对不同的股票i和不同的时间t，估计同一个期望函数g，并保证g在样本集外有最大的预测能力。






## 2. 样本集合划分和验证

1. 训练集
2. 验证集
3. 测试集

## 3. 几种主要的机器学习方法

### 3.1 简单线性模型

线性最小二乘法。对每一个因子引入权重$\theta_{i, t}$, $\theta = (\theta_{i, t})$。

$$
g(z_{i, t};\theta) = z^{T}_{i, t} \theta \tag{3}
$$

目标函数：

$$
   L(\theta) = \frac{1}{NT} \sum_1^{N}\sum_1^T (r_{i, t+1} - g(z_{i, t}; \theta))^2
$$

### 3.2 增强鲁棒性的线性模型

带权重的最小二乘:
$$
L(\theta) = \frac{1}{NT} \sum_1^{N}\sum_1^T w_{i, t}(r_{i, t+1} - g(z_{i, t}; \theta))^2  \tag{5}
$$

权重项$w_{i, t}$, 可以反比于某一个月的股票种类数。这样月份t对于损失的贡献是一样的。

也可以正比于证券i在时间t的市值。


使用Huber loss：

$$
L(\theta) = \frac{1}{NT} \Sigma_1^{N}\Sigma_1^T H((r_{i, t+1} - g(z_{i, t}; \theta)); \xi) \tag{6}
$$

where:
$$
H(X, \xi) = \begin{cases}
x^2& if  |x| <= \xi\\
 2\xi |x| - \xi^2&   if |x| > \xi
\end{cases}
$$

即Huber Loss在差异值小于$\xi$时，用的是平方误差，大于$\xi$时用的线性误差。HuberLoss针对outlier（特异点）具有较好的稳定性，不会因为特异点的存在使得loss function急剧增大。

### 3.3 带惩罚项的线性模型

惩罚因子对模型的复杂性进行惩罚，降低过拟合的可能性。

$$
L(\theta; .) = L(\theta) + \phi(\theta)  \tag{7}
$$



Elastic Net惩罚模型：
$$
\phi(\theta, \lambda, \rho) = \lambda (1 - \rho)\sum_{j=1}^{P}|\theta_j|  + \frac{1}{2}\lambda\rho\sum_{j=1}^P\theta_j^2 \tag{8}
$$

Elastic Net惩罚模型是说，综合使用了L1(Lasso)和L2(Ridge)的惩罚方式，当$\rho=0$时，就是Lasso回归， 当$\rho=1$时就是Ridge回归。


### 3.4 降维方法

### 3.5 广义线性模型

### 3.6 集成学习方法: Boosting Regression Trees和Random Forests